In [1]:
! pip install langchain
! pip install pinecone-client
! pip install pypdf
! pip install openai
! pip install tiktoken

   ---------------------------------------- 0.0/215.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/215.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/215.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/215.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/215.9 kB ? eta -:--:--
   ------- -------------------------------- 41.0/215.9 kB ? eta -:--:--
   ------- -------------------------------- 41.0/215.9 kB ? eta -:--:--
   ----------- ---------------------------- 61.4/215.9 kB 1.1 MB/s eta 0:00:01
   ----------- ---------------------------- 61.4/215.9 kB 1.1 MB/s eta 0:00:01
   ----------- ---------------------------- 61.4/215.9 kB 1.1 MB/s eta 0:00:01
   ------------------ ------------------- 102.4/215.9 kB 454.0 kB/s eta 0:00:01
   --------------------------- ---------- 153.6/215.9 kB 538.9 kB/s eta 0:00:01
   --------------------------- ---------- 153.6/215.9 kB 538.9 kB/s eta 0:00:01
   ---------------------

In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [3]:
!mkdir pdfs

In [4]:
loader=PyPDFDirectoryLoader("pdfs")

In [5]:
data=loader.load()

In [6]:
data[0]

Document(page_content='Compressing YOLOv7\nBenjamin van Zwienen\nMaster of Science Thesis\nNovember 24, 2023\nStudent Number: 4471180\nSupervisor: Prof. Dr. Ir. M. Wisse', metadata={'source': 'pdfs\\Thesis.pdf', 'page': 0})

In [9]:
data[0].page_content

'Compressing YOLOv7\nBenjamin van Zwienen\nMaster of Science Thesis\nNovember 24, 2023\nStudent Number: 4471180\nSupervisor: Prof. Dr. Ir. M. Wisse'

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [11]:
text_chunks = text_splitter.split_documents(data)

In [12]:
text_chunks

[Document(page_content='Compressing YOLOv7\nBenjamin van Zwienen\nMaster of Science Thesis\nNovember 24, 2023\nStudent Number: 4471180\nSupervisor: Prof. Dr. Ir. M. Wisse', metadata={'source': 'pdfs\\Thesis.pdf', 'page': 0}),
 Document(page_content='Abstract\nIn the literature, neural network compression can significantly reduce the number of floating-\npoint operations (FLOPs) of a neural network with limited accuracy loss. At the same\ntime, it is common to manually design smaller networks instead of using modern compres-\nsion techniques. This thesis will compare the two approaches for the object detection network\nYOLOv7. YOLOv7 can run in real time on a desktop GPU. F or edge GPUs a smaller version,', metadata={'source': 'pdfs\\Thesis.pdf', 'page': 2}),
 Document(page_content='called YOLOv7-tiny , was manually designed by the authors of YOLOv7. This thesis answers\nthe question: Can a state-of-the-art compression of YOLOv7 achieve higher accuracy than\nYOLOv7-tiny at the same numb

In [13]:
len(text_chunks)

284

In [14]:
print(text_chunks[0].page_content)

Compressing YOLOv7
Benjamin van Zwienen
Master of Science Thesis
November 24, 2023
Student Number: 4471180
Supervisor: Prof. Dr. Ir. M. Wisse


In [17]:
print(text_chunks[1].page_content)

real-time object detectors with 30 FPS or higher on GPU
V100. YOLOv7-E6 object detector (56 FPS V100, 55.9%
AP) outperforms both transformer-based detector SWIN-
L Cascade-Mask R-CNN (9.2 FPS A100, 53.9% AP) by
509% in speed and 2% in accuracy, and convolutional-
based detector ConvNeXt-XL Cascade-Mask R-CNN (8.6
FPS A100, 55.2% AP) by 551% in speed and 0.7% AP
in accuracy, as well as YOLOv7 outperforms: YOLOR,
YOLOX, Scaled-YOLOv4, YOLOv5, DETR, Deformable


In [15]:
print(text_chunks[2].page_content)

called YOLOv7-tiny , was manually designed by the authors of YOLOv7. This thesis answers
the question: Can a state-of-the-art compression of YOLOv7 achieve higher accuracy than
YOLOv7-tiny at the same number of floating-point operations?
First, two state-of-the-art compression methods are selected and compared on YOLOv7-
tiny . Then the best performing method, GBIP , is used to compress YOLOv7 till it has the
same number of FLOPs as YOLOv7-tiny . F rom the experiments it is determined that GBIP


In [17]:
print(text_chunks[283].page_content)

Processing Systems 32 , pp. 8024–8035, Curran Associates, Inc., 2019.
[73] L. N. Smith, “No more pesky learning rate guessing games,” CoRR , vol. abs/1506.01186,
2015.
55


In [18]:
import os
os.environ["OPENAI_API_KEY"]=""

In [54]:
embedding=OpenAIEmbeddings()

In [55]:
len(embedding.embed_query("How are you"))

1536

In [21]:
len(embedding.embed_query("hi i am fine"))

1536

In [56]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [62]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [27]:
import pinecone

In [57]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

In [31]:
index_name = "testing" # put in the name of your pinecone index here

In [58]:
index = pc.Index('testing')


## Create Embeddings for each of the Text Chunk

In [38]:
! pip install langchain_pinecone

In [42]:
for t in text_chunks:
    print(t.page_content)

Compressing YOLOv7
Benjamin van Zwienen
Master of Science Thesis
November 24, 2023
Student Number: 4471180
Supervisor: Prof. Dr. Ir. M. Wisse
Abstract
In the literature, neural network compression can significantly reduce the number of floating-
point operations (FLOPs) of a neural network with limited accuracy loss. At the same
time, it is common to manually design smaller networks instead of using modern compres-
sion techniques. This thesis will compare the two approaches for the object detection network
YOLOv7. YOLOv7 can run in real time on a desktop GPU. F or edge GPUs a smaller version,
called YOLOv7-tiny , was manually designed by the authors of YOLOv7. This thesis answers
the question: Can a state-of-the-art compression of YOLOv7 achieve higher accuracy than
YOLOv7-tiny at the same number of floating-point operations?
First, two state-of-the-art compression methods are selected and compared on YOLOv7-
tiny . Then the best performing method, GBIP , is used to compress YOLOv7 ti

In [43]:
docs = [t.page_content for t in text_chunks]

In [63]:
# docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)
from langchain_pinecone import PineconeVectorStore
# docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)
docsearch = PineconeVectorStore.from_documents(text_chunks, embedding, index_name=index_name)

In [64]:
docsearch

In [65]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001E69DB232C0>)

In [66]:
query= "YOLOv7 outperforms which models"

In [68]:
docs = docsearch.similarity_search(query)

In [39]:
docs

[Document(page_content='YOLOv7-tiny 6.2 3.5 320 30.8% 47.3% 32.2% 10.0% 31.9% 52.2%\nimprovement -39% -49% - = = = -0.9 = +0.7\nYOLOR-E6 [81] 115.8M 683.2G 1280 55.7% 73.2% 60.7% 40.1% 60.4% 69.2%\nYOLOv7-E6 97.2M 515.2G 1280 55.9% 73.5% 61.1% 40.6% 60.3% 70.0%\nimprovement -19% -33% - +0.2 +0.3 +0.4 +0.5 -0.1 +0.8\nYOLOR-D6 [81] 151.7M 935.6G 1280 56.1% 73.9% 61.2% 42.4% 60.5% 69.9%\nYOLOv7-D6 154.7M 806.8G 1280 56.3% 73.8% 61.4% 41.3% 60.6% 70.1%\nYOLOv7-E6E 151.7M 843.2G 1280 56.8% 74.4% 62.1% 40.8% 62.1% 70.6%'),
 Document(page_content='YOLOv5-L6 (r6.1) [23] 76.8M 445.6G 1280 63 - / 53.7% - -\nYOLOX-X [21] 99.1M 281.9G 640 58 51.5% / 51.1% - -\nYOLOv7-E6 97.2M 515.2G 1280 56 56.0% /55.9% 73.5% 61.2%\nYOLOR-E6 [81] 115.8M 683.2G 1280 45 55.8% / 55.7% 73.4% 61.1%\nPPYOLOE-X [85] 98.4M 206.6G 640 45 52.2% / 51.9% 69.9% 56.5%\nYOLOv7-D6 154.7M 806.8G 1280 44 56.6% /56.3% 74.0% 61.8%\nYOLOv5-X6 (r6.1) [23] 140.7M 839.2G 1280 38 - / 55.0% - -\nYOLOv7-E6E 151.7M 843.2G 1280 36 56.8% /56.8

In [69]:
llm = OpenAI()

c:\Users\alams\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [72]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())


In [73]:
query = "YOLOv7 outperforms which models"

In [74]:
qa.run(query)

c:\Users\alams\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' YOLOv7 outperforms YOLOv7-tiny and YOLOv7-W6, as well as the original YOLO network.'

In [1]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")